In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dense, Dropout, Reshape, Permute, concatenate, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras import preprocessing


In [ ]:
def DeXpression(input_shape=None,padding=None,classes=7):
    img_input = Input(shape=input_shape)


    conv_1 = Convolution2D(64, (7, 7), strides=(2, 2), padding=padding, activation='relu', name='conv_1')(img_input)
    maxpool_1 = MaxPooling2D((3, 3), strides=(2,2))(conv_1)
    x = BatchNormalization()(maxpool_1)
    
    # FEAT-EX1
    conv_2a = Convolution2D(96, (1, 1), strides=(1,1), activation='relu', padding=padding, name='conv_2a')(x)
    conv_2b = Convolution2D(208, (3, 3), strides=(1,1), activation='relu', padding=padding, name='conv_2b')(conv_2a)
    maxpool_2a = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_2a')(x)
    conv_2c = Convolution2D(64, (1, 1), strides=(1,1), name='conv_2c')(maxpool_2a)
    concat_1 = concatenate([conv_2b,conv_2c],axis=3,name='concat_2')
    maxpool_2b = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_2b')(concat_1)
    
  
    
    # FEAT-EX2
    conv_3a = Convolution2D(96, (1, 1), strides=(1,1), activation='relu', padding=padding, name='conv_3a')(maxpool_2b)
    conv_3b = Convolution2D(208, (3, 3), strides=(1,1), activation='relu', padding=padding, name='conv_3b')(conv_3a)
    maxpool_3a = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_3a')(maxpool_2b)
    conv_3c = Convolution2D(64, (1, 1), strides=(1,1), name='conv_3c')(maxpool_2a)
    concat_3 = concatenate([conv_3b,conv_3c],axis=3,name='concat_3')
    maxpool_3b = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_3b')(concat_3)
    

    net = Flatten()(maxpool_3b)
    net = Dense(classes, activation='softmax', name='predictions')(net)

    model = Model(inputs= img_input, outputs=net)
    return model

In [ ]:
model = DeXpression(input_shape=(48,48,1),padding='same',classes=7)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])
model.summary()

In [ ]:
TRAIN_DIR = "../input/ckplus/CK+48"
RESIZE_SHAPE = (48,48)
train_datagen = preprocessing.image.ImageDataGenerator(validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=RESIZE_SHAPE,
    batch_size=32,
    color_mode='grayscale',
    class_mode="categorical", 
    subset = 'training'
)

valid_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=RESIZE_SHAPE,
    batch_size=32,
    class_mode="categorical", 
    color_mode='grayscale',
    subset = 'validation'
)

In [ ]:
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath='../output/kaggle/working/weights.{epoch:02d}-{val_accuracy:.2f}.hdf5', 
    verbose=1, 
    monitor='val_accuracy',
    mode = 'max',
    save_weights_only=False, save_best_only=True )

model.fit_generator(generator=train_generator,steps_per_epoch=788/32, callbacks=[cp_callback],
	validation_data=valid_generator, validation_steps=193/32 , epochs=10, verbose=1)